In [7]:
import os
import pandas as pd
from collections import Counter
import re
import datetime
import time

start_time = time.time()

def extract_words(column_data):
    words_list = []
    for row in column_data:
        if isinstance(row, str):
            words = re.findall(r'\b\w{3,}\b', row)
            words_list.extend(words)
    return words_list

def find_best_identifier(process_df, process_file):
    words_list = extract_words(process_df['Réu'])

    # Conta a frequência de cada palavra na lista
    word_counter = Counter(words_list)
    most_common_words = word_counter.most_common()

    # Encontra o identificador com base na frequência das palavras
    file_words = re.findall(r'\b\w{3,}\b', process_file)

    for word, frequency in most_common_words:
        if frequency >= len(process_df) * 0.7 and word != "LTDA":
            return word, frequency

    for word in file_words:
        if word != "LTDA" and word in words_list:
            return word, frequency

    return "vazio", 0

# Verifica a frequência de um identificador específico no DataFrame
def check_identifier_match(process_df, identifier):
    words_list = []

    for row in process_df['Réu']:
        if isinstance(row, str):
            words = re.findall(r'\b\w{3,}\b', row)
            words_list.extend(words)

    word_counter = Counter(words_list)
    identifier_frequency = word_counter[identifier]
    return identifier_frequency

# Encontra o identificador, levando em conta o manual (se fornecido)
def find_identifier(process_df, process_file, manual_identifier=None):
    auto_identifier, auto_frequency = find_best_identifier(process_df, process_file)
    
    if manual_identifier:
        manual_frequency = check_identifier_match(process_df, manual_identifier)
        if manual_frequency >= len(process_df) * 0.7:
            identifier = manual_identifier
            frequency = manual_frequency
        else:
            identifier = auto_identifier
            frequency = auto_frequency
    else:
        identifier = auto_identifier
        frequency = auto_frequency

    return identifier, frequency

process_folder = "C:\\Teste\\Monitoramento de processos Kurier\\BASE_DE_PROCESSOS_KURIER"
process_files = [f for f in os.listdir(process_folder) if os.path.isfile(os.path.join(process_folder, f))]
all_process_dfs = []
no_identifier_dfs = []

# Obter os nomes das colunas da aba mais à direita da primeira planilha
first_file = process_files[0]
first_workbook = pd.read_excel(os.path.join(process_folder, first_file), sheet_name=None, header=None)
last_sheet_first_workbook = list(first_workbook.values())[-1]
column_names = last_sheet_first_workbook.iloc[2].tolist()

# Adicione um dicionário para mapear nomes de arquivos a identificadores manuais
manual_identifiers = {
    "RDOD_Mosaic__01.01.2014_16.02.2023_15022023110246 (1).xlsx": "MOSAIC"
}

for process_file in process_files:
    print(f"Processando arquivo: {process_file}")
    workbook = pd.read_excel(os.path.join(process_folder, process_file), sheet_name=None, header=None)
    sheet_dfs = []

    for sheet_name, sheet in workbook.items():
        sheet = sheet.iloc[3:]
        sheet.columns = column_names
        sheet_dfs.append(sheet)

    process_df = pd.concat(sheet_dfs, ignore_index=True)
    manual_identifier = manual_identifiers.get(process_file)
    identifier, frequency = find_identifier(process_df, process_file, manual_identifier)
    print(f"Identificador encontrado: {identifier} (Frequência: {frequency})")

    if manual_identifier:
        auto_identifier, auto_frequency = find_best_identifier(process_df, process_file)
        manual_frequency = check_identifier_match(process_df, manual_identifier)
        print(f"Comparação de frequência: Manual ({manual_identifier}: {manual_frequency}) vs. Automático ({auto_identifier}: {auto_frequency})")

    # Separa as linhas que não possuem o identificador e as armazena em um DataFrame separado
    no_identifier_rows = process_df[~process_df['Réu'].str.contains(identifier, na=False, case=False)]
    no_identifier_dfs.append(no_identifier_rows)

    # Remove as linhas que não possuem o identificador do DataFrame principal
    process_df = process_df[process_df['Réu'].str.contains(identifier, na=False, case=False)]
    process_df['Identificador'] = identifier
    all_process_dfs.append(process_df)

    combined_df = pd.concat(all_process_dfs, ignore_index=True)
    print("Número de linhas no DataFrame combinado:", len(combined_df))

    # Combina todos os DataFrames "Sem_Identificador" em um único DataFrame
    no_identifier_df = pd.concat(no_identifier_dfs, ignore_index=True)
    
    end_time = time.time()
duration = end_time - start_time
print(f"Tempo total de execução: {duration:.2f} segundos")

    


Processando arquivo: RDOD_Mosaic__01.01.2014_16.02.2023_15022023110246 (1).xlsx
Identificador encontrado: MOSAIC (Frequência: 2536)
Comparação de frequência: Manual (MOSAIC: 2536) vs. Automático (FERTILIZANTES: 2568)
Número de linhas no DataFrame combinado: 2509
Processando arquivo: RDOD_Raízen__01.01.2014_03.12.2022_02122022160505 (1).xlsx
Identificador encontrado: RAIZEN (Frequência: 18586)
Número de linhas no DataFrame combinado: 20973
Tempo total de execução: 6.86 segundos


In [8]:
# Verifica se as colunas 'Réu' e 'Identificador' estão presentes no DataFrame combinado
if 'Réu' not in combined_df.columns or 'Identificador' not in combined_df.columns:
    raise ValueError("As colunas 'Réu' e 'Identificador' devem estar presentes no DataFrame combinado.")

# Verifica se não há valores nulos ou vazios nas colunas 'Réu' e 'Identificador' do DataFrame combinado
if combined_df['Réu'].isnull().values.any() or combined_df['Réu'].str.strip().eq('').any():
    raise ValueError("A coluna 'Réu' não pode ter valores nulos ou vazios.")

if combined_df['Identificador'].isnull().values.any() or combined_df['Identificador'].str.strip().eq('').any():
    raise ValueError("A coluna 'Identificador' não pode ter valores nulos ou vazios.")
    
# Verifica se o número de linhas no DataFrame combinado é igual à soma dos números de linhas dos DataFrames individuais
if len(combined_df) != sum([len(df) for df in all_process_dfs]):
    raise ValueError("O número de linhas no DataFrame combinado deve ser igual à soma dos números de linhas dos DataFrames individuais.")
    
# Verifica se o número de linhas no DataFrame 'Sem_Identificador' é igual ao número de linhas removidas do DataFrame combinado
if len(no_identifier_df) != sum([len(df) for df in no_identifier_dfs]):
    raise ValueError("O número de linhas no DataFrame 'Sem_Identificador' deve ser igual ao número de linhas removidas do DataFrame combinado.")
    
# Verifica se todas as planilhas no arquivo Excel de entrada possuem as mesmas colunas e o mesmo cabeçalho
for process_file in process_files:
    print(f"Verificando arquivo: {process_file}")
    workbook = pd.read_excel(os.path.join(process_folder, process_file), sheet_name=None, header=None)
    last_sheet_first_workbook = list(workbook.values())[-1]
    if last_sheet_first_workbook.iloc[2].tolist() != column_names:
        raise ValueError(f"As planilhas no arquivo '{process_file}' devem ter as mesmas colunas e o mesmo cabeçalho.")
        
print("Verificação concluída com sucesso.")


Verificando arquivo: RDOD_Mosaic__01.01.2014_16.02.2023_15022023110246 (1).xlsx
Verificando arquivo: RDOD_Raízen__01.01.2014_03.12.2022_02122022160505 (1).xlsx
Verificação concluída com sucesso.


In [9]:
def find_active_clients(clients_df, identifiers):
    active_clients = []
    for identifier in identifiers:
        clients_with_identifier = clients_df[clients_df['Cliente'].str.contains(identifier, na=False, case=False)].copy()
        clients_with_identifier.loc[:, 'Identificador'] = identifier
        active_clients.append(clients_with_identifier)
    return pd.concat(active_clients, ignore_index=True)

# Encontre a planilha mais recente na pasta "C:\Teste\Monitoramento de processos Kurier\BASE_CLIENTES_ATIVOS_POR_ÁREA_E_TOUCH"
clients_folder = "C:\\Teste\\Monitoramento de processos Kurier\\BASE_CLIENTES_ATIVOS_POR_ÁREA_E_TOUCH"
client_files = [f for f in os.listdir(clients_folder) if os.path.isfile(os.path.join(clients_folder, f))]
client_file = sorted(client_files, key=lambda x: os.path.getmtime(os.path.join(clients_folder, x)))[-1]

# Leia a planilha e encontre a coluna "Cliente"
clients_df = pd.read_excel(os.path.join(clients_folder, client_file))

# Encontre os identificadores únicos presentes no DataFrame "combined_df"
unique_identifiers = combined_df['Identificador'].unique()

# Crie um novo DataFrame com os clientes ativos que contêm os identificadores
active_clients_df = find_active_clients(clients_df, unique_identifiers)

# Exporte todos os DataFrames como abas separadas no arquivo Excel
now = datetime.datetime.now().strftime("%d.%m.%y_%H.%M")
# Verifique se o arquivo já existe
filename = "C:\\Teste\\Monitoramento de processos Kurier\\BASE_BI\\Base_Kurier_Processos_BI.xlsx"
temp_filename = "C:\\Teste\\Monitoramento de processos Kurier\\BASE_BI\\Base_Kurier_Processos_BI_Temp.xlsx"

if os.path.exists(filename):
    try:
        os.remove(filename)
    except PermissionError:
        filename = temp_filename
else:
    filename = filename

# Adicionar a coluna "Origem" no DataFrame "no_identifier_df"
no_identifier_df["Origem"] = no_identifier_df["Identificador"]

# Exporte todos os DataFrames como abas separadas no arquivo Excel
with pd.ExcelWriter(filename) as writer:
    combined_df.to_excel(writer, sheet_name="Identificados", index=False)
    no_identifier_df.to_excel(writer, sheet_name="Sem_Identificador", index=False)
    active_clients_df.to_excel(writer, sheet_name="Base_Clientes_Ativos", index=False)

print(f"Arquivo exportado como: {filename}")


KeyError: 'Identificador'

In [ ]:
# Carregar a planilha "Base_Kurier_Processos_BI.xlsx" e os DataFrames de clientes e processos
clients_df = pd.read_excel(r"C:\Teste\Monitoramento de processos Kurier\BASE_BI\Base_Kurier_Processos_BI.xlsx", sheet_name='Base_Clientes_Ativos')
process_df = pd.read_excel(r"C:\Teste\Monitoramento de processos Kurier\BASE_BI\Base_Kurier_Processos_BI.xlsx", sheet_name='Identificados')

# Função para remover espaços em branco depois do último dígito de uma string
def remove_trailing_spaces(s):
    return re.sub(r'\s+\Z', '', s)

# Função para determinar se um processo é "Próprio" (pertence ao cliente) ou não
def is_own_process(client_name, cell):
    max_char_difference = 3
    
    # Remover espaços em branco depois do último dígito das strings
    client_name = remove_trailing_spaces(client_name)
    cell = remove_trailing_spaces(cell)
    
    # Verificar a diferença no número de caracteres
    char_difference = abs(len(client_name) - len(cell))
    
    if char_difference <= max_char_difference:
        # Contar a quantidade de caracteres diferentes entre as strings
        char_mismatch_count = sum(1 for a, b in zip(client_name, cell) if a != b)
        
        if char_mismatch_count <= max_char_difference:
            return True
    return False

def get_client_name(identifier, clients_df):
    client_row = clients_df[clients_df['Cliente'].str.contains(identifier, na=False, case=False)].iloc[0]
    return client_row['Cliente']

def determine_process_type(row, client_name):
    if is_own_process(client_name, row['Réu']):
        return "Próprio"
    else:
        return "Terceiro"

# Processar o DataFrame de processos e adicionar a coluna 'Tipo_Processo'
identifier = process_df['Identificador'].iloc[0]
client_name = get_client_name(identifier, clients_df)
process_df['Tipo_Processo'] = process_df.apply(determine_process_type, axis=1, args=(client_name,))

# Salvar as alterações na planilha "Base_Kurier_Processos_BI.xlsx"
with pd.ExcelWriter(r"C:\Teste\Monitoramento de processos Kurier\BASE_BI\Base_Kurier_Processos_BI.xlsx", mode='a') as writer:
    clients_df.to_excel(writer, sheet_name='Base_Clientes_Ativos', index=False)
    process_df.to_excel(writer, sheet_name='Identificados', index=False)
